In [6]:
#install kaggle
#!pip install kaggle


In [7]:
!mkdir .kaggle

mkdir: cannot create directory ‘.kaggle’: File exists


In [8]:
import json
token = {"username":"itsasif","key":"01d92bea1a9c994aa63b102e0439a334"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [9]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [10]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [11]:
!chmod 600 /root/.kaggle/kaggle.json

In [12]:
#search for the dataset
!kaggle datasets list -s planets-dataset

ref                                               title                                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
emirhanai/planets-and-moons-dataset-ai-in-space   Planets and Moons Dataset - AI in Space 🌌 🛰 ☄ 🔭   46MB  2023-06-17 04:17:35            687         39  1.0              
eduardowoj/exoplanets-database                    Exoplanets Database                              281KB  2018-02-28 20:56:05           1538         60  0.8235294        
mrisdal/open-exoplanet-catalogue                  Open Exoplanet Catalogue                         123KB  2017-06-08 19:59:22           3933        169  0.8235294        
iamsouravbanerjee/planet-dataset                  Planet Dataset                                     2KB  2023-07-17 06:37:25           2301     

In [13]:
#download dataset
!kaggle datasets download -d nikitarom/planets-dataset -p /content

100% 1.50G/1.50G [01:20<00:00, 21.5MB/s]
100% 1.50G/1.50G [01:20<00:00, 20.1MB/s]


In [14]:
#unzip the dataset
%%capture
!unzip \*.zip

In [15]:
#Import necessary libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import fbeta_score
from tqdm import tqdm
import cv2
from PIL import Image
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import fbeta_score
import time
%matplotlib inline

pal = sns.color_palette()

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls


In [16]:
#check present working directory
! pwd

/content


In [17]:
#read the train classes csv file and save a copy
df_train_classes= pd.read_csv('/content/planet/planet/train_classes.csv')
df_train = df_train_classes.copy()
df_train.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [18]:
#read the sample submission classes csv file and save a copy
df_test_classes = pd.read_csv('/content/planet/planet/sample_submission.csv')
df_test = df_test_classes.copy()
df_test.head()

,image_name,tags
0,test_0,primary clear agriculture road water
1,test_1,primary clear agriculture road water
2,test_2,primary clear agriculture road water
3,test_3,primary clear agriculture road water
4,test_4,primary clear agriculture road water


In [19]:
#total train data points
!ls /content/planet/planet/train-jpg | wc -l


40479


In [20]:
test1 = !ls /content/planet/planet/test-jpg | wc -l
test2 = !ls /content/test-jpg-additional/test-jpg-additional | wc -l
assert df_test.shape[0] == float(test1[0])+float(test2[0])

In [21]:
#list of unique labels/tags
labels = df_train['tags'].apply(lambda x: x.split(' '))
from collections import Counter, defaultdict
counts = defaultdict(int)
for l in labels:
    for l2 in l:
        counts[l2] += 1

tag_list=list(counts.keys())
tag_list

['haze',
 'primary',
 'agriculture',
 'clear',
 'water',
 'habitation',
 'road',
 'cultivation',
 'slash_burn',
 'cloudy',
 'partly_cloudy',
 'conventional_mine',
 'bare_ground',
 'artisinal_mine',
 'blooming',
 'selective_logging',
 'blow_down']

In [22]:
#total unique labels
len(tag_list)

17

In [23]:
#assign numerical value to each label using dictionary
label_map = {i:j for j, i in enumerate(tag_list)}
label_map

{'haze': 0,
 'primary': 1,
 'agriculture': 2,
 'clear': 3,
 'water': 4,
 'habitation': 5,
 'road': 6,
 'cultivation': 7,
 'slash_burn': 8,
 'cloudy': 9,
 'partly_cloudy': 10,
 'conventional_mine': 11,
 'bare_ground': 12,
 'artisinal_mine': 13,
 'blooming': 14,
 'selective_logging': 15,
 'blow_down': 16}

In [24]:
# One hot encode the training labels. Convert the images into pixels and resize them
X_train, Y_train = [], []
for img, label in tqdm(df_train.values, miniters = 1000):
  target = np.zeros(17)
  for tag in label.split(' '):
    target[label_map[tag]]=1
  X_train.append(cv2.resize(cv2.imread('/content/planet/planet/train-jpg/{}.jpg'.format(img)), (64,64)))
  Y_train.append(target)

100%|██████████| 40479/40479 [00:54<00:00, 744.37it/s]


In [25]:
#convert the test images to pixels and resize them as well
X_test=[]
for img, label in tqdm(df_test[:40669].values, miniters = 1000):
  X_test.append(cv2.resize(cv2.imread('/content/planet/planet/test-jpg/{}.jpg'.format(img)), (64,64)))
for img, label in tqdm(df_test[40669:].values, miniters = 1000):
  X_test.append(cv2.resize(cv2.imread('/content/test-jpg-additional/test-jpg-additional/{}.jpg'.format(img)), (64,64)))
x_test = np.array(X_test, np.float16)/255

100%|██████████| 20522/20522 [00:27<00:00, 751.04it/s]


In [26]:
#Confirm the dimensions
len(X_test), len(X_train), len(Y_train)

(61191, 40479, 40479)

In [27]:
import sys
sys.getsizeof(X_train)

351064

In [28]:
#Change lists to numpy arrays and normalize
x_train = np.array(X_train, np.float16)/255
y_train = np.array(Y_train, np.uint8)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, shuffle = True, random_state = 1)

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

(32383, 64, 64, 3) (32383, 17) (8096, 64, 64, 3) (8096, 17)


In [29]:
#delete X_test, X_train, Y_train to free up space
del(X_test, X_train, Y_train)

In [30]:
#Split the data into 5 folds and train on 4 folds while validating on 1 fold

yfull_test = []
yfull_train = []
X_train_, X_val_, Y_train_, Y_val_ = train_test_split(x_train, y_train, test_size = 0.2, random_state = 1)

In [31]:
#Build a CNN model
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import VGG16
from keras.optimizers import Adam, SGD

input_size = 64
input_channels = 3
batch_size = 128

# Add model parameters including dropout, layers and activation function
base_model = VGG19(include_top=False,
                   weights='imagenet',
                   input_shape=(input_size, input_size, input_channels))

model = Sequential()
model.add(BatchNormalization(input_shape=(input_size, input_size, input_channels)))

model.add(base_model)
model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(17, activation='sigmoid'))

80134624/80134624 [==============================] - 4s 0us/step


In [32]:
# define model training optimizer parameters
opt  = SGD(lr=0.1)
model.compile(loss='binary_crossentropy',optimizer=opt, metrics=['accuracy'])
callbacks = [EarlyStopping(monitor='val_loss', patience=2, verbose=0),
                ModelCheckpoint(filepath='weights/best_weights',
                                 save_best_only=True,
                                 save_weights_only=True)]
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 64, 64, 3)        12        
 ormalization)                                                   
                                                                 
 vgg19 (Functional)          (None, 2, 2, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 17)                34833     
                                                                 
Total params: 20,059,229
Trainable params: 20,059,223
Non-trainable params: 6
____________________________________________

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



In [33]:
gc.collect()

1294

In [34]:
#Try a combination of epoch lengths and learning rates

model.fit(x = X_train_, y= Y_train_, validation_data=(X_val_, Y_val_),batch_size=64,verbose=2, epochs=10,callbacks=callbacks,shuffle=True)

Epoch 1/10
405/405 - 57s - loss: 0.1568 - accuracy: 0.7211 - val_loss: 0.1291 - val_accuracy: 0.6688 - 57s/epoch - 142ms/step
Epoch 2/10
405/405 - 42s - loss: 0.1181 - accuracy: 0.7136 - val_loss: 0.1083 - val_accuracy: 0.7732 - 42s/epoch - 104ms/step
Epoch 3/10
405/405 - 41s - loss: 0.1081 - accuracy: 0.7369 - val_loss: 0.1140 - val_accuracy: 0.8055 - 41s/epoch - 101ms/step
Epoch 4/10
405/405 - 41s - loss: 0.1014 - accuracy: 0.7233 - val_loss: 0.1091 - val_accuracy: 0.7591 - 41s/epoch - 102ms/step


In [35]:
p_val = model.predict(X_val_, batch_size = 64, verbose=2)
print(fbeta_score(Y_val_, np.array(p_val) > 0.2, beta=2, average='samples')) #Check the model performance on the validation set

102/102 - 3s - 3s/epoch - 28ms/step
0.9038346993788665


In [36]:
#import and initialise Garbage Collector
import gc
gc.collect()


1493

In [37]:
df_test.to_csv('asif_Submission.csv', index = False) #Export to csv
df_test.head()

,image_name,tags
0,test_0,primary clear agriculture road water
1,test_1,primary clear agriculture road water
2,test_2,primary clear agriculture road water
3,test_3,primary clear agriculture road water
4,test_4,primary clear agriculture road water


In [38]:
#download submission file
from google.colab import files
files.download('asif_Submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>